## NB very simple model for testing

In [5]:
%pip install xgboost
%pip install pandas
%pip install matplotlib
%pip install scikit-learn
%pip install seaborn

Active code page: 1252
Note: you may need to restart the kernel to use updated packages.
Active code page: 1252
Note: you may need to restart the kernel to use updated packages.
Active code page: 1252
Note: you may need to restart the kernel to use updated packages.
Active code page: 1252
  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/37/a9/08538c29f188b1c37eb6e115a5046c6d17457e1a7b8ea748edd3017ddd6c/scikit_learn-1.3.1-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for threadpoolctl>=2.0.0 from https://files.pythonhosted.org/packages/81/12/fd4dea011af9d69e1cad05c75f3f7202cdcbeac9b712eea58ca779a72865/threadpoolctl-3.2.0-py3-none-any.whl.metadata
  Using cached threadpoolctl-3.2.0-py3-none-any.whl.metadata (10.0 kB)
   ---------------------------------------- 0.0/9.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.1 MB 2.0 MB/s eta 0:00:05
   - -------------------------------------- 0.4/9.1 MB 6.4

In [6]:
%matplotlib inline
import numpy as np, pandas as pd, seaborn as sns, matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit
import joblib
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error


In [7]:
data = pd.read_csv('final.csv')

In [8]:
data.head(8)

,Date,Time,Player_count
0,9/24/2023,0:00,2711248
1,9/24/2023,0:05,2726267
2,9/24/2023,0:10,2712687
3,9/24/2023,0:15,2745816
4,9/24/2023,0:20,2756843
5,9/24/2023,0:25,2768107
6,9/24/2023,0:30,2777797
7,9/24/2023,0:35,2786147


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3755 entries, 0 to 3754
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Date          3755 non-null   object
 1   Time          3755 non-null   object
 2   Player_count  3755 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 88.1+ KB


In [10]:
data = data.drop(data[(data["Player_count"]== 0)].index)

In [11]:
data["New_date"] = pd.to_datetime(data['Date'], format='%m/%d/%Y')

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3606 entries, 0 to 3754
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          3606 non-null   object        
 1   Time          3606 non-null   object        
 2   Player_count  3606 non-null   int64         
 3   New_date      3606 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 140.9+ KB


In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3606 entries, 0 to 3754
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          3606 non-null   object        
 1   Time          3606 non-null   object        
 2   Player_count  3606 non-null   int64         
 3   New_date      3606 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 140.9+ KB


### Feature engineering (transform data to be used for the machine learning algo) 

In [14]:

data["New_time"] = pd.to_datetime(data['Time'], format='%H:%M')

In [15]:
data["year"] = data["New_date"].dt.year
data["month"] = data["New_date"].dt.month
data["day"] = data["New_date"].dt.day
data["hour"] = data["New_time"].dt.hour
data["min"] = data["New_time"].dt.minute

In [16]:
data.drop(columns=["Date","Time","New_date","New_time"], inplace=True)

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3606 entries, 0 to 3754
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   Player_count  3606 non-null   int64
 1   year          3606 non-null   int32
 2   month         3606 non-null   int32
 3   day           3606 non-null   int32
 4   hour          3606 non-null   int32
 5   min           3606 non-null   int32
dtypes: int32(5), int64(1)
memory usage: 126.8 KB


In [18]:
data.head()

,Player_count,year,month,day,hour,min
0,2711248,2023,9,24,0,0
1,2726267,2023,9,24,0,5
2,2712687,2023,9,24,0,10
3,2745816,2023,9,24,0,15
4,2756843,2023,9,24,0,20


### One hot encoding

### Preapering for training 

### Models and testing

#### xgboost

In [19]:
tss = TimeSeriesSplit(n_splits=5, test_size=11, gap=11)
df = data.sort_index()


fold = 0
preds = []
scores = []
for train_idx, val_idx in tss.split(df):
    train = df.iloc[train_idx]
    test = df.iloc[val_idx]

    FEATURES = ['year', 'month', 'day', 'hour', 'min']
    TARGET = 'Player_count'

    X_train = train[FEATURES].values
    y_train = train[TARGET]

    X_test = test[FEATURES].values
    y_test = test[TARGET]

    reg = xgb.XGBRegressor(base_score=0.5, booster='gbtree',    
                           n_estimators=1000,
                           early_stopping_rounds=50,
                           objective='reg:linear',
                           max_depth=3,
                           learning_rate=0.01)
    reg.fit(X_train, y_train,
            eval_set=[(X_train, y_train), (X_test, y_test)],
            verbose=100)

    y_pred = reg.predict(X_test)
    preds.append(y_pred)
    score = np.sqrt(mean_squared_error(y_test, y_pred))
    scores.append(score)

[0]	validation_0-rmse:1755891.00177	validation_1-rmse:2523757.63771
[100]	validation_0-rmse:692442.83417	validation_1-rmse:1188393.30562
[200]	validation_0-rmse:328761.13229	validation_1-rmse:673221.07893
[300]	validation_0-rmse:212850.41783	validation_1-rmse:394853.85739


c:\Users\Tommy\Desktop\4410-ruby-acorn\.venv\Lib\site-packages\xgboost\data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
c:\Users\Tommy\Desktop\4410-ruby-acorn\.venv\Lib\site-packages\xgboost\core.py:160: UserWarning: [04:10:35] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[400]	validation_0-rmse:167324.73106	validation_1-rmse:315610.68452
[500]	validation_0-rmse:144080.32090	validation_1-rmse:251400.59437
[600]	validation_0-rmse:128392.75038	validation_1-rmse:202173.49539


[700]	validation_0-rmse:118013.85830	validation_1-rmse:167056.83433
[800]	validation_0-rmse:110645.42035	validation_1-rmse:138714.75588
[900]	validation_0-rmse:105461.34231	validation_1-rmse:122013.44319
[999]	validation_0-rmse:101020.71212	validation_1-rmse:108608.92195
[0]	validation_0-rmse:1759899.18610	validation_1-rmse:2337510.96722
[100]	validation_0-rmse:694605.19080	validation_1-rmse:977049.37694
[200]	validation_0-rmse:330646.36790	validation_1-rmse:438950.55034
[300]	validation_0-rmse:214235.76404	validation_1-rmse:142009.06173
[400]	validation_0-rmse:168163.85490	validation_1-rmse:60049.54139

c:\Users\Tommy\Desktop\4410-ruby-acorn\.venv\Lib\site-packages\xgboost\data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
c:\Users\Tommy\Desktop\4410-ruby-acorn\.venv\Lib\site-packages\xgboost\core.py:160: UserWarning: [04:10:36] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


...

[400]	validation_0-rmse:168344.32067	validation_1-rmse:132327.57953
[500]	validation_0-rmse:145570.76987	validation_1-rmse:73183.63783
[600]	validation_0-rmse:129918.31189	validation_1-rmse:64982.57692
[615]	validation_0-rmse:128037.01715	validation_1-rmse:68271.07487
[0]	validation_0-rmse:1764796.67694	validation_1-rmse:2925794.40547
[100]	validation_0-rmse:697024.76886	validation_1-rmse:1364886.91514
[200]	validation_0-rmse:331321.81371	validation_1-rmse:678962.36314
[300]	validation_0-rmse:214049.44184	validation_1-rmse:344775.70925


c:\Users\Tommy\Desktop\4410-ruby-acorn\.venv\Lib\site-packages\xgboost\data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
c:\Users\Tommy\Desktop\4410-ruby-acorn\.venv\Lib\site-packages\xgboost\core.py:160: UserWarning: [04:10:36] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[400]	validation_0-rmse:168519.06602	validation_1-rmse:224612.11990
[500]	validation_0-rmse:145761.71394	validation_1-rmse:130262.26115
[600]	validation_0-rmse:130448.76574	validation_1-rmse:67357.71155
[700]	validation_0-rmse:120550.13767	validation_1-rmse:35493.38280
[756]	validation_0-rmse:115880.25964	validation_1-rmse:41133.97686
[0]	validation_0-rmse:1767771.96906	validation_1-rmse:3105092.09726
[100]	validation_0-rmse:698165.54292	validation_1-rmse:1265338.78964
[200]	validation_0-rmse:331669.34456	validation_1-rmse:582271.38736
[300]	validation_0-rmse:213845.42739	validation_1-rmse:292886.13203


c:\Users\Tommy\Desktop\4410-ruby-acorn\.venv\Lib\site-packages\xgboost\data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
c:\Users\Tommy\Desktop\4410-ruby-acorn\.venv\Lib\site-packages\xgboost\core.py:160: UserWarning: [04:10:37] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[400]	validation_0-rmse:168366.59743	validation_1-rmse:169780.72950
[500]	validation_0-rmse:145899.98016	validation_1-rmse:92580.82607
[600]	validation_0-rmse:130350.26398	validation_1-rmse:36465.99526
[700]	validation_0-rmse:120296.46737	validation_1-rmse:21783.44280
[723]	validation_0-rmse:118330.30008	validation_1-rmse:27148.44715


In [20]:
print(f'Score across folds {np.mean(scores):0.4f}')
print(f'Fold scores:{scores}')

Score across folds 51395.3164
Fold scores:[108568.87661209588, 33429.3839504532, 61137.55498882306, 34721.963745294495, 19118.80253373269]


Saving model to pkl file for later use


In [21]:
reg.save_model("reg_model.json")
